In [81]:
import os
import datetime
import json
import sqlite3

In [119]:
for root,directories, files in os.walk('../nni'):
    for file in files:
        file_name = '{}/{}'.format(root,file)

In [120]:
def parse_state(file_name: str):
    with open(file_name, 'r') as f:
        time_stamp = int(f.readlines()[0].split(' ')[-1]) / 1000.0
        exp_time = datetime.datetime.fromtimestamp(time_stamp)
        return exp_time
def parse_metrics(file_name: str):
    with open(file_name, 'r') as f:
        content = f.read()
        file_head = content.split('{')[0]
        metric_details =  json.loads(content.replace(file_head, ''))
        return {
            'metric_id':file_head,
            **metric_details
        }
def parse_parameters(file_name: str):
    with open(file_name, 'r') as f:
        content = f.read()
        return json.loads(content)
def parse_sqlite(file_name:str):
    try:
        conn = sqlite3.connect(file_name)
        c = conn.cursor()
        c.execute(' SELECT * FROM ExperimentProfile')
        result = c.fetchall()
    finally:
        conn.close()
    return json.loads(result[0][0])

In [121]:
file_name = '../nni/experiments/MrUaVQmO/db/nni.sqlite'
meta_data = file_name.split('/')
experiment_id = meta_data[3]
trial_id = meta_data[5]
file_type = meta_data[-1]
parse_dict = {
    'state': parse_state,
    'metrics': parse_metrics,
    'parameter.cfg': parse_parameters,
    'nni.sqlite': parse_sqlite,
}
parse_dict[file_type](file_name)


{'authorName': 'Xiao HE',
 'experimentName': 'tunning_model_1',
 'trialConcurrency': 1,
 'maxExecDuration': 14400,
 'maxTrialNum': 30,
 'searchSpace': '{"hidden_factors": {"_type": "choice", "_value": [4, 8, 16, 32, 64, 128]}, "batch": {"_type": "choice", "_value": [128, 256, 512, 1024, 2048]}, "regularizer": {"_type": "uniform", "_value": [-8, -4]}}',
 'trainingServicePlatform': 'local',
 'advisor': {'builtinAdvisorName': 'BOHB',
  'classArgs': {'optimize_mode': 'maximize',
   'min_budget': 1,
   'max_budget': 27,
   'eta': 3}},
 'versionCheck': True,
 'clusterMetaData': [{'key': 'codeDir',
   'value': '/root/nni-experiments/model_1/.'},
  {'key': 'command', 'value': 'python main.py'}]}